In [1]:
# Load Some Packages
import numpy as np
import imageio
import scipy as sp
from scipy import signal
import matplotlib.pyplot as plt
import seaborn as sns
import os, glob, ntpath
from tqdm import tqdm

In [2]:
# Given Some Coefficients in Partial(K)/Partial(t)
D = 0.75

# Given Some Coefficients in F(K)
A = -0.3
Kr = 0.03
Ktheta = 0.2
Km = 1.0

# Given Some Coefficients in Partial(r)/Partial(t)
B = 0.0001
C = 10.0

# max & min in colorbar
vmax = 1.0
vmin = 0.0

In [3]:
# Given the range of the rectangle
lenX = 200
lenY = 200

deltaX = 0.1
deltaY = 0.1
deltaT = (deltaX*deltaY)**2/(deltaX**2 + deltaY**2)/2/D
D*deltaT

0.0025000000000000005

In [4]:
# Definie the filter of the laplacian operator
filter_laplacian = np.array([[1/deltaX/deltaY, 1/deltaY**2, 1/deltaX/deltaY],
                           [1/deltaX**2, -4/deltaX/deltaY - 2/deltaX**2 - 2/deltaY**2, 1/deltaX**2],
                           [1/deltaX/deltaY, 1/deltaY**2, 1/deltaX/deltaY]])

filter_laplacian = np.array([[0, 1/deltaY**2, 0],
                           [1/deltaX**2, - 2/deltaX**2 - 2/deltaY**2, 1/deltaX**2],
                           [0, 1/deltaY**2, 0]])

filter_laplacian

array([[   0.,  100.,    0.],
       [ 100., -400.,  100.],
       [   0.,  100.,    0.]])

In [5]:
def Dirichlet_bd(maxIter, clamping_value, clamping_radius,
                 clamping_ticks, clamping_ticks_end,
                 foucs_loc,
                 K, Kt, R, Rt,
                 top, bottom, left, right):
    
    center_point_loc = int(lenX/2)
    center_point_loc = np.vstack((center_point_loc, int(lenY/2)))
    point_value = np.zeros([maxIter, foucs_loc.shape[0]])
    R_value = np.zeros([maxIter, foucs_loc.shape[0]])
    
    # Fixed the boundary values
    K[:,0] = left
    K[:,-1] = right
    K[0,:] = top
    K[-1,:] = bottom
    
    for i in tqdm(range(maxIter)):
        
        if i >= clamping_ticks and i <= clamping_ticks_end:  
            r2 = clamping_radius**2
            for a in range(lenX):
                for b in range(lenY):
                    dis2 = (a-center_point_loc[0])**2 + (b-center_point_loc[1])**2
                    if dis2 < r2:
                        K[a,b] = clamping_value

        # Fick's second law
        R = R + deltaT*Rt
        Kt = D*sp.signal.convolve2d(K, filter_laplacian, mode = 'same', boundary='symm') + A*(K - Kr)*(K - Ktheta)*(K - Km)*(K + 0.1) - R*K
        K = K + deltaT*Kt

        # Fixed the boundary values
        K[:,0] = left
        K[:,-1] = right
        K[0,:] = top
        K[-1,:] = bottom

        # plot the images
        fig, ax = plt.subplots()
        im = ax.imshow(K, cmap='hot', vmin=vmin,vmax=vmax)
        ax.set_axis_off()
        ax.set_title('{:.1f} ms'.format(i*deltaT*1000))
        fig.colorbar(im, cax=fig.add_axes(), orientation='vertical')
        plt.savefig('../images/Diri/{}.png'.format(i))
        plt.clf()
        
        # Fick's second law
        Rt = B*((K-Kr) - C*R)
        
        for j in range(foucs_loc.shape[0]):
            point_value[i,j] = K[foucs_loc[j,0],foucs_loc[j,1]]
            R_value[i,1] = R[foucs_loc[j,0],foucs_loc[j,1]]
        
    return point_value, R_value

In [6]:
def Dirichlet_bd_only_diff(maxIter, clamping_value, clamping_radius,
                 clamping_ticks, clamping_ticks_end,
                 foucs_loc,
                 K, Kt,
                 top, bottom, left, right):
    
    center_point_loc = int(lenX/2)
    center_point_loc = np.vstack((center_point_loc, int(lenY/2)))
    point_value = np.zeros([maxIter, foucs_loc.shape[0]])
    
    # Fixed the boundary values
    K[:,0] = left
    K[:,-1] = right
    K[0,:] = top
    K[-1,:] = bottom
    
    for i in tqdm(range(maxIter)):
        
        if i >= clamping_ticks and i <= clamping_ticks_end:  
            r2 = clamping_radius**2
            for a in range(lenX):
                for b in range(lenY):
                    dis2 = (a-center_point_loc[0])**2 + (b-center_point_loc[1])**2
                    if dis2 < r2:
                        K[a,b] = clamping_value

        # Fick's second law
        Kt = D*sp.signal.convolve2d(K, filter_laplacian, mode = 'same', boundary='symm')
        K = K + deltaT*Kt

        # Fixed the boundary values
        K[:,0] = left
        K[:,-1] = right
        K[0,:] = top
        K[-1,:] = bottom
        
        for j in range(foucs_loc.shape[0]):
            point_value[i,j] = K[foucs_loc[j,0],foucs_loc[j,1]]
        
    return point_value

In [7]:
def Dirichlet_bd_with_fK(maxIter, clamping_value, clamping_radius,
                 clamping_ticks, clamping_ticks_end,
                 foucs_loc,
                 K, Kt,
                 top, bottom, left, right):
    
    center_point_loc = int(lenX/2)
    center_point_loc = np.vstack((center_point_loc, int(lenY/2)))
    point_value = np.zeros([maxIter, foucs_loc.shape[0]])
    
    # Fixed the boundary values
    K[:,0] = left
    K[:,-1] = right
    K[0,:] = top
    K[-1,:] = bottom
    
    for i in tqdm(range(maxIter)):
        
        if i >= clamping_ticks and i <= clamping_ticks_end:  
            r2 = clamping_radius**2
            for a in range(lenX):
                for b in range(lenY):
                    dis2 = (a-center_point_loc[0])**2 + (b-center_point_loc[1])**2
                    if dis2 < r2:
                        K[a,b] = clamping_value

        # Fick's second law
        Kt = D*sp.signal.convolve2d(K, filter_laplacian, mode = 'same', boundary='symm') + A*(K - Kr)*(K - Ktheta)*(K - Km)*(K + 0.1)
        K = K + deltaT*Kt

        # Fixed the boundary values
        K[:,0] = left
        K[:,-1] = right
        K[0,:] = top
        K[-1,:] = bottom
        
        for j in range(foucs_loc.shape[0]):
            point_value[i,j] = K[foucs_loc[j,0],foucs_loc[j,1]]
        
    return point_value

In [8]:
def Neumann_bd( maxIter,
                clamping_value, clamping_radius,
                clamping_ticks, clamping_ticks_end,
                foucs_loc,
                K, Kt, R, Rt,
                top, bottom, left, right):
    
    center_point_loc = int(lenX/2)
    center_point_loc = np.vstack(center_point_loc, int(lenY/2))
    point_value = np.zeros([maxIter, foucs_loc.shape[0]])
    R_value = np.zeros([maxIter, foucs_loc.shape[0]])
    
    
    for i in tqdm(range(maxIter)):

        if i >= clamping_ticks and i <= clamping_ticks_end:  
            r2 = clamping_radius**2
            for a in range(lenX):
                for b in range(lenY):
                    dis2 = (a-center_point_loc[0])**2 + (b-center_point_loc[1])**2
                    if dis2 < r2:
                        K[a,b] = clamping_value
        
        # Fick's second law
        R = R + deltaT*Rt

        Kt = sp.signal.convolve2d(K, filter_laplacian, mode = 'same')
        Kt = D*Kt + A*(K - Kr)*(K - Ktheta)*(K - Km)*(K + 0.1) - R*K
        K = K + deltaT*Kt
        
        
        # Fixed the differential boundary values
        Kt[:,0] = left
        Kt[:,-1] = right
        Kt[0,:] = top
        Kt[-1,:] = bottom
        
        
        # plot the images
        fig, ax = plt.subplots()
        im = ax.imshow(K, cmap='hot', vmin=vmin,vmax=vmax)
        ax.set_axis_off()
        ax.set_title('{:.1f} ms'.format(i*deltaT*1000))
        fig.colorbar(im, cax=fig.add_axes(), orientation='vertical')
        plt.savefig('../images/Neum/{}.png'.format(i))
        plt.clf()

        # Fick's second law
        Rt = B*((K-Kr) - C*R)

        point_value[i,:] = K[foucs_loc[:,0],foucs_loc[:,1]]
        R_value[i,1] = R[foucs_loc[:,0],foucs_loc[:,1]]
        
    return point_value, R_value

In [9]:
def Periodic_bd( maxIter,
                 clamping_value, clamping_radius,
                 clamping_ticks, clamping_ticks_end,
                 foucs_loc,
                 K, Kt, R, Rt):
    
    center_point_loc = int(lenX/2)
    center_point_loc = np.vstack(center_point_loc, int(lenY/2))
    point_value = np.zeros([maxIter, foucs_loc.shape[0]])
    R_value = np.zeros([maxIter, foucs_loc.shape[0]])
    
    
    for i in tqdm(range(maxIter)):

        if i >= clamping_ticks and i <= clamping_ticks_end:  
            r2 = clamping_radius**2
            for a in range(lenX):
                for b in range(lenY):
                    dis2 = (a-center_point_loc[0])**2 + (b-center_point_loc[1])**2
                    if dis2 < r2:
                        K[a,b] = clamping_value
        
        # Fick's second law
        R = R + deltaT*Rt
        Kt = D*sp.signal.convolve2d(K, filter_laplacian, mode = 'same', boundary = 'wrap') + A*(K - Kr)*(K - Ktheta)*(K - Km)*(K + 0.1) - R*K
        K = K + deltaT*Kt
        
        # plot the images
        fig, ax = plt.subplots()
        im = ax.imshow(K, cmap='hot', vmin=vmin,vmax=vmax)
        ax.set_axis_off()
        ax.set_title('{:.1f} ms'.format(i*deltaT*1000))
        fig.colorbar(im, cax=fig.add_axes(), orientation='vertical')
        plt.savefig('../images/Perid/{}.png'.format(i))
        plt.clf()
        # Fick's second law
        Rt = B*((K-Kr) - C*R)

        center_point_value[i,0] = i
        center_point_value[i,1] = K[int(center_point_loc[0,0]),int(center_point_loc[1,0])]
        R_value[i,0] = i
        R_value[i,1] = R[int(center_point_loc[0,0]),int(center_point_loc[1,0])]
        
    return center_point_value, R_value

In [10]:
def compute_fick_law(boundary, maxIter,
                     clamping_value, clamping_radius, clamping_ticks, clamping_ticks_end, 
                     foucs_loc, K, Kt, R, Rt, 
                     top = 0.0, bottom = 0.0, left = 0.0, right = 0.0):
    '''N: The number of the random points you want to pick'''
    '''clamping_ticks: '''
    
    # remove other images in the dir
    for image in sorted(glob.glob('./images/*.png'), key=lambda k: int(ntpath.basename(k).replace('.png', ''))):
        os.remove(image)
    
    if boundary == 'Diri':
        point_value, R_value = Dirichlet_bd(maxIter,
                                            clamping_value, clamping_radius, clamping_ticks, clamping_ticks_end,
                                            foucs_loc, K, Kt, R, Rt, 
                                            top, bottom, left, right)
        
    elif boundary == 'Neum':
        point_value, R_value = Neumann_bd(maxIter, 
                                          clamping_value, clamping_radius, clamping_ticks, clamping_ticks_end, 
                                          foucs_loc, K, Kt, R, Rt, 
                                          top, bottom, left, right)
        
    elif boundary == 'Perid':
        point_value, R_value = Periodic_bd(maxIter, 
                                           clamping_value, clamping_radius, clamping_ticks, clamping_ticks_end, 
                                           foucs_loc, K, Kt, R, Rt)
            
    return point_value, R_value

In [11]:
def make_video(images, image_folder, video_name, outimg=None, fps=5, size=None,
               is_color=True, format="XVID"):
    
    from cv2 import VideoWriter, VideoWriter_fourcc, imread, resize
    fourcc = VideoWriter_fourcc(*format)
    vid = None
    for image in images:
        if not os.path.join(image_folder, image):
            raise FileNotFoundError(image)
        img = imread(os.path.join(image_folder, image))
        if vid is None:
            if size is None:
                size = img.shape[1], img.shape[0]
            vid = VideoWriter(video_name, fourcc, float(fps), size, is_color)
        if size[0] != img.shape[1] and size[1] != img.shape[0]:
            img = resize(img, size)
        vid.write(img)
    vid.release()
    return vid

In [12]:
def do_timestep(u0, u, r0):
    # Propagate with forward-difference in time, central-difference in space

    u[1:-1, 1:-1] = u0[1:-1, 1:-1] + D * deltaT * (
          (u0[2:, 1:-1] - 2*u0[1:-1, 1:-1] + u0[:-2, 1:-1])/deltaX/deltaX
          + (u0[1:-1, 2:] - 2*u0[1:-1, 1:-1] + u0[1:-1, :-2])/deltaY/deltaY ) + deltaT*A*(
        u0[1:-1, 1:-1]-Kr)*(u0[1:-1, 1:-1]-Ktheta)*(u0[1:-1, 1:-1]-Km)*(u0[1:-1, 1:-1]+0.1) - r0*u[1:-1, 1:-1]

    u[:,:2] = Kr
    u[:,-2:] = Kr
    u[:2,:] = Kr
    u[-2:,:] = Kr


    r0 = r0 + deltaT*B*((u0[1:-1, 1:-1] - Kr) - C*r0)

    u0 = u.copy()

    return u0, u, r0

In [13]:
def Dirichlet_noconv(maxIter,
                     clamping_value, clamping_radius,
                     clamping_ticks, clamping_ticks_end,
                     foucs_loc,
                     K, R):

    # set the initial value    
    r0 = R
    k0 = np.empty([K.shape[0]+2, K.shape[1]+2])
    k = np.zeros([K.shape[0]+2, K.shape[1]+2])
    
    center_point_loc = int(K.shape[0]/2)
    center_point_loc = np.vstack((center_point_loc, int(K.shape[1]/2)))
    point_value = np.zeros([maxIter, foucs_loc.shape[0]])
    R_value = np.zeros([maxIter, foucs_loc.shape[0]])

    k0[1:-1,1:-1] = K
    k0[1:-1,0], k0[1:-1,-1] = K[:,0], K[:,-1]
    k0[0,1:-1], k0[1,1:-1] = K[0,:], K[-1,:]
    k0[0,0], k0[0,-1], k0[-1,0], k0[-1,-1] = K[0,0], K[0,-1], K[-1,0], K[-1,-1]
    
    for i in tqdm(range(maxIter)):
        if i >= clamping_ticks and i <= clamping_ticks_end:
            for a in range(k0.shape[0]):
                for b in range(k0.shape[1]):
                    r2 = 9
                    dis2 = (a-center_point_loc[0,0])**2 + (b-center_point_loc[1,0])**2
                    if dis2 < r2:
                        k0[a+1,b+1] = clamping_value

        k0, k, r0 = do_timestep(k0, k, r0)
        
        for j in range(foucs_loc.shape[0]):
            point_value[i,j] = k0[foucs_loc[j,0]+1,foucs_loc[j,1]+1]
            R_value[i,1] = r0[foucs_loc[j,0],foucs_loc[j,1]]
        
    return point_value, R_value

In [14]:
def do_timestep_only_diff(u0, u):
    # Propagate with forward-difference in time, central-difference in space

    u[1:-1, 1:-1] = u0[1:-1, 1:-1] + D * deltaT * (
          (u0[2:, 1:-1] - 2*u0[1:-1, 1:-1] + u0[:-2, 1:-1])/deltaX/deltaX
          + (u0[1:-1, 2:] - 2*u0[1:-1, 1:-1] + u0[1:-1, :-2])/deltaY/deltaY )
    
    u[:,:2] = Kr
    u[:,-2:] = Kr
    u[:2,:] = Kr
    u[-2:,:] = Kr

    u0 = u.copy()

    return u0, u

In [15]:
def Dirichlet_noconv_only_diff(maxIter,
                     clamping_value, clamping_radius,
                     clamping_ticks, clamping_ticks_end,
                     foucs_loc,
                     K):

    # set the initial value    
    k0 = np.empty([K.shape[0]+2, K.shape[1]+2])
    k = np.zeros([K.shape[0]+2, K.shape[1]+2])
    
    center_point_loc = int(K.shape[0]/2)
    center_point_loc = np.vstack((center_point_loc, int(K.shape[1]/2)))
    point_value = np.zeros([maxIter, foucs_loc.shape[0]])

    k0[1:-1,1:-1] = K
    k0[1:-1,0], k0[1:-1,-1] = K[:,0], K[:,-1]
    k0[0,1:-1], k0[1,1:-1] = K[0,:], K[-1,:]
    k0[0,0], k0[0,-1], k0[-1,0], k0[-1,-1] = K[0,0], K[0,-1], K[-1,0], K[-1,-1]
    
    for i in tqdm(range(maxIter)):
        if i >= clamping_ticks and i <= clamping_ticks_end:
            for a in range(k0.shape[0]):
                for b in range(k0.shape[1]):
                    r2 = 9
                    dis2 = (a-center_point_loc[0,0])**2 + (b-center_point_loc[1,0])**2
                    if dis2 < r2:
                        k0[a+1,b+1] = clamping_value

        k0, k= do_timestep_only_diff(k0, k)
        
        for j in range(foucs_loc.shape[0]):
            point_value[i,j] = k0[foucs_loc[j,0]+1,foucs_loc[j,1]+1]
        
    return point_value

In [16]:
def do_timestep_with_fK(u0, u):
    # Propagate with forward-difference in time, central-difference in space

    u[1:-1, 1:-1] = u0[1:-1, 1:-1] + D * deltaT * (
          (u0[2:, 1:-1] - 2*u0[1:-1, 1:-1] + u0[:-2, 1:-1])/deltaX/deltaX
          + (u0[1:-1, 2:] - 2*u0[1:-1, 1:-1] + u0[1:-1, :-2])/deltaY/deltaY )+ deltaT*A*(
        u0[1:-1, 1:-1]-Kr)*(u0[1:-1, 1:-1]-Ktheta)*(u0[1:-1, 1:-1]-Km)*(u0[1:-1, 1:-1]+0.1)
    
    u[:,:2] = Kr
    u[:,-2:] = Kr
    u[:2,:] = Kr
    u[-2:,:] = Kr

    u0 = u.copy()

    return u0, u

In [17]:
def Dirichlet_noconv_with_fK(maxIter,
                     clamping_value, clamping_radius,
                     clamping_ticks, clamping_ticks_end,
                     foucs_loc,
                     K):

    # set the initial value    
    k0 = np.empty([K.shape[0]+2, K.shape[1]+2])
    k = np.zeros([K.shape[0]+2, K.shape[1]+2])
    
    center_point_loc = int(K.shape[0]/2)
    center_point_loc = np.vstack((center_point_loc, int(K.shape[1]/2)))
    point_value = np.zeros([maxIter, foucs_loc.shape[0]])

    k0[1:-1,1:-1] = K
    k0[1:-1,0], k0[1:-1,-1] = K[:,0], K[:,-1]
    k0[0,1:-1], k0[1,1:-1] = K[0,:], K[-1,:]
    k0[0,0], k0[0,-1], k0[-1,0], k0[-1,-1] = K[0,0], K[0,-1], K[-1,0], K[-1,-1]
    
    for i in tqdm(range(maxIter)):
        if i >= clamping_ticks and i <= clamping_ticks_end:
            for a in range(k0.shape[0]):
                for b in range(k0.shape[1]):
                    r2 = 9
                    dis2 = (a-center_point_loc[0,0])**2 + (b-center_point_loc[1,0])**2
                    if dis2 < r2:
                        k0[a+1,b+1] = clamping_value

        k0, k= do_timestep_with_fK(k0, k)
        
        for j in range(foucs_loc.shape[0]):
            point_value[i,j] = k0[foucs_loc[j,0]+1,foucs_loc[j,1]+1]
        
    return point_value